In [17]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer


df = pd.read_csv('property_and_building_violations.csv')


descriptions = df['description'].dropna().astype(str).str.lower()


vectorizer = CountVectorizer(ngram_range=(3, 3), stop_words='english')
X = vectorizer.fit_transform(descriptions)


sum_words = X.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)


freq_df = pd.DataFrame(words_freq[:50], columns=['trigram', 'count'])
print(freq_df)


                        trigram  count
0         failure obtain permit   4006
1          failed comply permit    444
2            comply permit term    444
3         failure secure permit    421
4          building use premise    285
5               use premise req    285
6       protection adj property    264
7           use premises permit     65
8        electrical work permit     62
9     mechanical execution work     60
10     maintenance means egress     42
11      emergency escape rescue     40
12          guarding live parts     38
13           failed comply prmt     38
14            comply prmt terms     38
15          mass state plumbing     30
16          state plumbing code     30
17            exits exit access     25
18            number exits exit     24
19     illegally occupied prior     23
20            acts 1956 amended     21
21              stop work order     20
22      accessible means egress     18
23  municipal bylaws ordinances     17
24          permits perfo

In [20]:
import pandas as pd


df = pd.read_csv('property_and_building_violations.csv')


def categorize_violation(desc):
    if pd.isna(desc):
        return 'Unknown'
    desc = desc.upper()
    if 'PERMIT' in desc or 'COMPLY' in desc:
        return 'Permit Issues'
    elif 'MAINTAIN' in desc:
        return 'Property Maintenance Issue'
    elif 'UNSAFE' in desc or 'DANGEROUS' in desc:
        return 'Unsafe Conditions'
    elif 'ELECTRICAL' in desc or 'WIRING' in desc:
        return 'Electrical Violation'
    elif 'PREMISE' in desc:
        return 'Premise Violation'
    elif 'CERTIFICATION' in desc:
        return 'Certification Issues'
    elif 'WATER' in desc or 'PLUMBING' in desc:
        return 'Water Violation'
    else:
        return 'Other'


df['violation_category'] = df['description'].apply(categorize_violation)


df['ward_str'] = df['ward'].astype(str).str.strip()
df = df[df['ward_str'].str.isdigit()]


agg = df.groupby(['violation_category', 'ward_str'], as_index=False).agg(count=('case_no', 'count'))


csv_path = 'violation_counts_by_category_ward.csv'
agg.to_csv(csv_path, index=False)


print("Preview: Counts by Category and Ward", agg.head(20))

print(f"CSV file saved to: {csv_path}")


Preview: Counts by Category and Ward       violation_category ward_str  count
0   Certification Issues       01     98
1   Certification Issues       02      7
2   Certification Issues       03    320
3   Certification Issues       04     35
4   Certification Issues       05    174
5   Certification Issues       06     21
6   Certification Issues       07     20
7   Certification Issues       08     15
8   Certification Issues       09     38
9   Certification Issues       10      9
10  Certification Issues       11      5
11  Certification Issues       12      9
12  Certification Issues       13     13
13  Certification Issues       14     17
14  Certification Issues       15      4
15  Certification Issues       16      2
16  Certification Issues       17      1
17  Certification Issues       18     31
18  Certification Issues       19      2
19  Certification Issues       20      2
CSV file saved to: violation_counts_by_category_ward.csv


CHECK OUT THE FLOURISH VISUALIZATIONS HERE: https://public.flourish.studio/visualisation/22902450/

In [26]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier



viol = pd.read_csv(
    'property_and_building_violations.csv',
    usecols=['case_no','sam_id','status','status_dttm','description'],
    parse_dates=['status_dttm'],
    nrows=5000
)


sam = pd.read_csv(
    'SAM_ID.csv',
    usecols=['SAM_ADDRESS_ID','FULL_ADDRESS','created_date'],
    parse_dates=['created_date']
)


def categorize_violation(desc):
    if pd.isna(desc): return 'Other'
    d = desc.upper()
    if 'PERMIT' in d or 'COMPLY' in d: return 'Permit Issues'
    if 'MAINTAIN' in d: return 'Property Maintenance'
    if 'UNSAFE' in d or 'DANGEROUS' in d: return 'Unsafe Conditions'
    if 'ELECTRICAL' in d or 'WIRING' in d: return 'Electrical'
    if 'PREMISE' in d: return 'Premise'
    if 'CERTIFICATION' in d: return 'Certification'
    if 'WATER' in d or 'PLUMBING' in d: return 'Water'
    return 'Other'

viol['violation_cat'] = viol['description'].apply(categorize_violation)


viol = viol.merge(
    sam[['SAM_ADDRESS_ID','created_date','FULL_ADDRESS']],
    left_on='sam_id', right_on='SAM_ADDRESS_ID',
    how='left'
)


now = datetime.now()
agg1 = viol.groupby('sam_id').agg(
    total_violations=('case_no','count'),
    days_since_last=('status_dttm', lambda ts: (now - ts.max()).days),
    days_since_sam_created=('created_date', lambda ds: (now - ds.max()).days)
)
cat_counts = viol.pivot_table(
    index='sam_id', columns='violation_cat', values='case_no', aggfunc='count', fill_value=0
)
features = agg1.join(cat_counts, how='left').fillna(0)
features['scofflaw'] = (
    (features.total_violations>=6) |
    (viol.groupby('sam_id')['status'].apply(lambda s: (s=='Open').any()))
).astype(int)


X = features.drop(columns='scofflaw')
y = features['scofflaw']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=42)
pipe = Pipeline([('imp',SimpleImputer(strategy='median')),('sc',StandardScaler()),('rfc',RandomForestClassifier(n_estimators=20,random_state=42))])
pipe.fit(X_train,y_train)

#predicting
features['risk_score'] = pipe.predict_proba(X)[:,1]
results = features[['risk_score','total_violations']].reset_index().merge(
    sam[['SAM_ADDRESS_ID','FULL_ADDRESS']],
    left_on='sam_id', right_on='SAM_ADDRESS_ID', how='left'
)

# 8) Display top 10
top10 = results.sort_values('risk_score', ascending=False)
print("Top 10 High-Risk Properties (Sample)", top10[['FULL_ADDRESS','sam_id','total_violations','risk_score']])

# 9) Save CSV
csv_path = 'risk_scores_sample.csv'
results.to_csv(csv_path, index=False)
csv_path


Top 10 High-Risk Properties (Sample)                FULL_ADDRESS    sam_id  total_violations  risk_score
1749  543 Massachusetts Ave   92356.0                 1         1.0
31        471-473 Albany St    1541.0                 1         1.0
398          548 E Broadway   21002.0                 1         1.0
1138          54 Gardner St   62619.0                 3         1.0
2422         40 Rosemont St  118489.0                 1         1.0
...                     ...       ...               ...         ...
1025          11 Fayette St   56953.0                 1         0.0
1023         65 Farragut Rd   56534.0                 1         0.0
1022        107 Farquhar St   56406.0                 1         0.0
1021    47-53 Farnsworth St   56399.0                 1         0.0
1003    41-49 Fairmount Ave   55744.0                 1         0.0

[3640 rows x 4 columns]


'risk_scores_sample.csv'

In [ ]:
import pandas as pd

# risk scores and their wards
prop = pd.read_csv('risk_scores_sample.csv')
sam  = pd.read_csv('SAM_ID.csv', usecols=['SAM_ADDRESS_ID','WARD'], dtype={'WARD':str})

merged = prop.merge(sam, left_on='sam_id', right_on='SAM_ADDRESS_ID', how='left')
ward_risk = merged.groupby('WARD', as_index=False).agg(avg_risk=('risk_score','mean'))
ward_risk.to_csv('risk_scores_by_ward.csv', index=False)


View VISUALIZATIONS HERE: https://public.flourish.studio/visualisation/22902951/